In [ ]:
# 유사도 관련 패키지
# !pip install fasttext

In [2]:
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# db 연동
import pymysql

# 주식 데이터
import FinanceDataReader as fdr

# nlp
from konlpy.tag import Okt

# 유사도 계산
from gensim import models

In [3]:
def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()
    
    # ============== 날짜 조정
    ## 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
    for i in range(len(df)):
        if df['date_h'].iloc[i] in after_market:
            df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for date in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][date] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
    return df_1

In [17]:
def giro_dic(df,num = 0):
    
    df = set_data(df, num)
    print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t 삭제
    df['text'] = df['text'].str.replace('[\n|\t|\r]','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')
    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        temp = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                temp.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = temp
    print('명사, 형용사 추출')
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
    media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    print('불용어 사전 로딩..')
    
    
    for k in range(len(df['Contents_input'])):
        for word in df['Contents_input'][k]:
            if word in stopwords:
                df['Contents_input'][k].remove(word)
            if word in media:
                df['Contents_input'][k].remove(word) 
    print('불용어 처리 완료')
    
    # 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for stop in wordList:
            if len(stop) <= 1:
                df['Contents_input'][index].remove(stop)
    print('단어 길이 1이하인 단어 제거..')
    
    
    # 주식 감성 사전 로딩 : koself
    koself_pro = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/KOSELF_pos.txt'
    koself_neg = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/KOSELF_neg.txt'
    
    positive = list(pd.read_csv(koself_pro, header=None)[0])
    negative = list(pd.read_csv(koself_neg, header=None)[0])
    print('koself 사전 로딩...')
    
    # koself 만 쓴 감성점수
    df['koself'] = 0
    for ix, wList in enumerate(df['Contents_input']):
        for ko in wList:
            if ko in positive:
                df['koself'][ix] += 1
            if ko in negative:
                df['koself'][ix] -= 1
    print('koself 점수 계산 완료')
    
    # 사전 구축
    # 유사도 계산으로 긍부정 단어 추가
    # 유사도 계산으로 애매한 단어 긍부정 판단--> 긍부정 단어에 다포함되어있는
    # 사용 모델 -->  fastText의 이미 학습된 한국어 모델 사용
#     try:
#         print(ko_model, '모델 로딩 안해도 됨')
#     except:
#         ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')
    print(ko_model, '모델 로딩 안해도 됨')
    
    # 단어 수를 늘리기 위해서 유사도 50%이상인 단어를 추가
    for p in positive:
        pos_si = [si_p[0] for si_p in ko_model.wv.similar_by_word(p, 10) if si_p[1] >= 0.5]
    for n in negative:
        neg_si = [si_n[0] for si_n in ko_model.wv.similar_by_word(n, 10) if si_n[1] >= 0.5]
    print('koself 유사도 계산 및 추가 완료...')
    
    pos_gi = positive + pos_si
    neg_gi = negative + neg_si
    
    
    for iex, ud in enumerate(df['ud']):
        if ud == 1: # positive?
            for sent in df['Contents_input'][iex]:
                if (sent in pos_gi) and (sent in neg_gi):
                    # 10개의 유사한 단어에서 positive가 5 이상이면 positive로 판단
                    S_word = [si_sent[0] for si_sent in ko_model.wv.similar_by_word(sent, 10)]
                    word_num = 0
                    for si_word in S_word:
                        if si_word in pos_gi:
                            word_num += 1
                        if word_num >= 5:
                            pos_gi.append(sent)
                            break
                else:
                    pos_gi.append(sent)
                    
        elif ud == -1:
            for sent_n in df['Contents_input'][iex]:
                if (sent_n in pos_gi) and (sent_n in neg_gi):
                    # 10개의 유사한 단어에서 negative가 5 이상이면 negative로 판단
                    S_word = [si_sent_n[0] for si_sent_n in ko_model.wv.similar_by_word(sent_n, 10)]
                    word_num = 0
                    for si_word2 in S_word:
                        if si_word2 in neg_gi:
                            word_num += 1
                        if word_num >= 5:
                            neg_gi.append(sent_n)
                            break
                else:
                    neg_gi.append(sent_n)      
        else:
            continue
            
    print('사전 구축 완료..')
    
    # 사전 중복 제거
    pos_gi = list(set(pos_gi))
    neg_gi = list(set(neg_gi))
    
    # 만들어진 사전으로 점수 도출
    df['giro'] = 0
    for Index, newList in enumerate(df['Contents_input']):
        for new_word in newList:
            if new_word in pos_gi:
                df['giro'][Index] += 1
            if new_word in neg_gi:
                df['giro'][Index] -= 1
    print('새로만든 사전으로 점수 계산 완료')
    
    return df, pos_gi, neg_gi

In [18]:
def giro_test(df,num = 0):

    df = set_data(df, num)
    print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t 삭제
    df['text'] = df['text'].str.replace('[\n|\t|\r]','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')
    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        temp = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                temp.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = temp
    print('명사, 형용사 추출')
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
    url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
    media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    print('불용어 사전 로딩..')
    
    
    for k in range(len(df['Contents_input'])):
        for word in df['Contents_input'][k]:
            if word in stopwords:
                df['Contents_input'][k].remove(word)
            if word in media:
                df['Contents_input'][k].remove(word) 
    print('불용어 처리 완료')
    
    # 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for stop in wordList:
            if len(stop) <= 1:
                df['Contents_input'][index].remove(stop)
    print('단어 길이 1이하인 단어 제거..')
    print('test 데이터 전처리 완료')
    return df

In [19]:
df = pd.read_csv('./df_매일경제_삼성전자.csv')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   st_n    15411 non-null  object
 1   st_cd   15411 non-null  int64 
 2   news    15411 non-null  object
 3   date    15411 non-null  int64 
 4   title   15411 non-null  object
 5   url     15411 non-null  object
 6   text    15411 non-null  object
dtypes: int64(2), object(5)
memory usage: 842.9+ KB


In [8]:
df['title'][0]

'"반도체 핵심소재 日의존도 90%"…삼성·하이닉스 대형 악재'

In [9]:
df.head()

,st_n,st_cd,news,date,title,url,text
0,삼성전자,5930,매일경제,2019063018,"""반도체 핵심소재 日의존도 90%""…삼성·하이닉스 대형 악재",http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n\n▶ 여기를 누르시면 크게 보실 수 있습니다\n 일본 정부가 반도체와 ...
1,삼성전자,5930,매일경제,2019063018,화학물질 투명성 높인 삼성전자,http://news.mk.co.kr/newsRead.php?year=2019&no...,삼성전자가 생산 현장에서 사용되는 소재와 화학물질에 대한 투명성을 대폭 강화한 것으...
2,삼성전자,5930,매일경제,2019062608,"삼성전자, 버블워시 탄생 11주년 기념 세탁기 보상판매",http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n[사진 제공 : 삼성전자]\n삼성전자는 '버블워시' 탄생 11주년을 기념...
3,삼성전자,5930,매일경제,2019062508,"삼성전자, `셰프컬렉션 인덕션` 쿠킹클래스 열어",http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n지난 24일 샘표 우리맛 공간에서 열린 삼성전자 셰프컬렉션 인덕션 쿠킹클...
4,삼성전자,5930,매일경제,2019062717,삼성전자·기아차 등 31곳…동반성장지수 `최우수`,http://news.mk.co.kr/newsRead.php?year=2019&no...,"삼성전자, 기아자동차 등 31개사가 2018년도 동반성장지수 평가에서 '최우수' 등..."


In [11]:
# train set
prt=df[:10]

In [9]:
# 로컬에서 할 때
# import os
# # 모델 크기가 크기 때문에 다른 곳에 저장__git에 커밋되지 않게
# print('현재 위치 : ', os.getcwd()) # 현재 디렉토리 위치
# os.chdir("../.git/비공개")
# print('이동 위치 : ',os.getcwd())

현재 위치 :  C:\Users\j\stock_predict\감성분석
이동 위치 :  C:\Users\j\stock_predict\.git\비공개


In [12]:
# 페이스북 한국어 임베딩 모델 다운 --> 1번만 하면 됨
import fasttext
import fasttext.util
fasttext.util.download_model('ko', if_exists='ignore')

'cc.ko.300.bin'

In [14]:
# 미리 모델 로드
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')
print(ko_model)

FastText(vocab=2000000, vector_size=300, alpha=0.025)


In [15]:
pred, pos_gi, neg_gi = giro_dic(prt)
pred.head()

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
불용어 사전 로딩..
불용어 처리 완료
단어 길이 1이하인 단어 제거..
koself 사전 로딩...
koself 점수 계산 완료
FastText(vocab=2000000, vector_size=300, alpha=0.025) 모델 로딩 안해도 됨
koself 유사도 계산 및 추가 완료...
사전 구축 완료..
새로만든 사전으로 점수 계산 완료


,st_n,st_cd,news,date,title,url,text,date_d,date_h,ud,Contents_input,koself,giro
0,삼성전자,5930,매일경제,2019061817,AI 핵심 `NPU`에 미래 건 삼성전자,http://news.mk.co.kr/newsRead.php?year=2019&no...,삼성전자가인공지능AI에활용되는차세대반도체인NPUNeuralProcessingUnit...,2019-06-19,17,1,"[인공, 지능, 활용, 차세대, 반도체, 신경망, 처리장치, 시장, 선점, 위해, ...",2,196
1,삼성전자,5930,매일경제,2019062111,"LG 가전, 美 신뢰도 평가서 세계 4위…삼성은 18위",http://news.mk.co.kr/newsRead.php?year=2019&no...,컨슈머리포트신뢰도조사사진출처컨슈머리포트홈페이지캡처LG가전이미국유력소비자매체가평가한신...,2019-06-21,11,1,"[컨슈머, 리포트, 신뢰, 사진, 출처, 컨슈머, 리포트, 홈페이지, 캡처, 가전,...",0,60
2,삼성전자,5930,매일경제,2019062417,"이재용 ""중동 脫석유 전략서 삼성 경쟁력 살려야""",http://news.mk.co.kr/newsRead.php?year=2019&no...,이재용삼성전자부회장오른쪽셋째이24일서울상일동삼성물산을찾아경영진과전략회의를진행한후구내...,2019-06-25,17,1,"[이재용, 부회장, 오른쪽, 비롯, 중동, 사업, 전략, 수립, 사우디, 실세, 왕...",2,210
3,삼성전자,5930,매일경제,2019062508,"삼성전자, `셰프컬렉션 인덕션` 쿠킹클래스 열어",http://news.mk.co.kr/newsRead.php?year=2019&no...,지난24일샘표우리맛공간에서열린삼성전자셰프컬렉션인덕션쿠킹클래스에참여한이충후셰프사진제공...,2019-06-25,08,1,"[지난, 인덕션, 화구, 최대, 개, 분할, 해사, 용하다, 높다, 효율, 조리, ...",0,37
4,삼성전자,5930,매일경제,2019062608,"삼성전자, 버블워시 탄생 11주년 기념 세탁기 보상판매",http://news.mk.co.kr/newsRead.php?year=2019&no...,사진제공삼성전자삼성전자는버블워시탄생11주년을기념해다음달31일까지세탁기보상판매를실시한...,2019-06-26,08,1,"[사진, 성전, 성전, 버블, 탄생, 주년, 기념, 음달, 용량, 드럼세탁기, 신제...",1,47


In [12]:
# os.chdir("C:/Users/j/stock_predict/감성분석")
# print('이동 위치 : ',os.getcwd())

In [70]:
pos_txt = ','.join(pos_gi).replace(',','\n')
neg_txt = ','.join(neg_gi).replace(',','\n')

In [71]:
# 결과 저장

pred.to_csv('./result/pred_300.csv', index=False)

with open('./result/pos_300.txt','w') as f:
    f.write(pos_txt)
with open('./result/neg_300.txt', 'w') as f:
    f.write(neg_txt)

#### train --> ud 정확도 판단

In [21]:
# 구축된 사전 단어 수
len(pos_gi), len(neg_gi)

(670, 350)

In [22]:
(len(pred[(pred.koself > 0) & (pred.ud > 0)])+len(pred[(pred.koself < 0) & (pred.ud < 0)])) / (len(pred[pred.ud > 0])+len(pred[pred.ud < 0]))

0.5

In [23]:
(len(pred[(pred.giro > 0) & (pred.ud > 0)]) + len(pred[(pred.giro < 0) & (pred.ud <0)])) / (len(pred[pred.ud > 0])+len(pred[pred.ud < 0]))

1.0

#### test --> 정확도 판단

In [75]:
test = df[12000:]

In [43]:
# giro_test(test)

okt 실행 완료
태그 삭제 완료
품사태깅 완료
명사, 형용사 추출 완료
불용어 사전 로딩 완료
불용어 처리 완료
단어 길이 1이하 단어 제거
test 데이터 전처리 완료


,st_n,st_cd,news,date,title,url,text,date_d,date_h,ud,Contents_input
0,삼성전자,5930,매일경제,2018010106,미디어 세상을 접수한 유튜브를 파헤치다,http://premium.mk.co.kr/view.php?no=21031,경제경영BookCafe블루오션시프트김위찬르네마보안비즈니스북스블루오션전략이후12년4차...,2018-01-02,06,1,"[경제, 경영, 블루, 오션, 시프트, 김위찬, 네마, 보안, 비즈니스, 북스, 블..."
1,삼성전자,5930,매일경제,2018010117,기반산업 따라 희비 갈린 부동산,http://news.mk.co.kr/newsRead.php?year=2018&no...,한때미분양의늪으로통하던경기도파주시부동산시장이달라지고있다지난해6월분양한파주문산역동문굿...,2018-01-02,17,1,"[미분, 통, 경기도, 파주시, 부동산, 시장, 지난해, 여명, 수도권, 광역급행,..."
2,삼성전자,5930,매일경제,2018010100,"동원산업·휠라코리아 ""생큐 트럼프""",http://news.mk.co.kr/newsRead.php?year=2018&no=30,2018법인세인하수혜올해1월부터법인세인하를골자로한미국세제개혁법안이현실화하면서국내증시...,2018-01-02,00,1,"[법인세, 인하, 수혜, 올해, 법인, 세율, 대폭, 핵심, 국외, 자금, 자국, ..."
3,삼성전자,5930,매일경제,2018010104,"[2018 국내 업종별 전망] 반도체, D램 독주 계속…`반도체 정점론` 부담",http://news.mk.co.kr/newsRead.php?year=2018&no...,인공지능AI사물인터넷IoT등4차산업혁명수요폭발에따른슈퍼사이클로한국경제의효자역할을톡톡...,2018-01-02,04,1,"[인공, 지능, 사물인터넷, 산업혁명, 수요, 폭발, 슈퍼, 사이클, 국경, 제의,..."
4,삼성전자,5930,매일경제,2018010104,[2018 경제기상도] `1월효과` 기대 커진 증시…반도체·바이오株 계속 달릴듯,http://news.mk.co.kr/newsRead.php?year=2018&no...,지난해연말조정장세는올해초코스피와코스닥시장1월효과의기폭제가될전망이다상장사들의탄탄한실적...,2018-01-02,04,1,"[지난해, 연말, 정장, 해초, 코스피, 코스닥시장, 돌파, 코스닥, 하이닉스, 정..."
...,...,...,...,...,...,...,...,...,...,...,...
3406,삼성전자,5930,매일경제,2021091216,세계 최대 삼성전자 프리미엄 가전 매장 통했다…10년만에 신규 백화점 갤러리아 광교...,http://news.mk.co.kr/newsRead.php?no=879535&ye...,갤러리아광교에자리잡은삼성프리미엄스토어내부모습사진제공갤러리아지난해3월갤러리아가10년만...,2021-09-13,16,1,"[갤러리아, 광교, 자리, 삼성, 프리미엄, 스토어, 내부, 모습, 사진, 제공, ..."
3407,삼성전자,5930,매일경제,2021091214,"""10만원에 페이스북 아마존 애플 넷플릭스 구글 다 살 수 있다?""",http://news.mk.co.kr/newsRead.php?no=878988&ye...,고가주식쪼개서매매아마존알파벳LG생활건강등주당100만원이훌쩍넘는고가우량주를커피한잔값에...,2021-09-13,14,1,"[가주, 매매, 아마존, 알파벳, 생활, 건강, 등주, 대의, 주식시장, 참여, 대..."
3408,삼성전자,5930,매일경제,2021091206,"펜트하우스 천서진 물로 씻은 그 폰, 갤Z플립3 수심 3m 수영장에 빠트렸더니…",http://news.mk.co.kr/newsRead.php?no=878229&ye...,천서진이피묻은갤럭시Z플립3를물로씻고작동하는모습사진출처유튜브캡처드라마펜트하우스3주인공...,2021-09-13,06,1,"[서진, 갤럭시, 플립, 를물, 작동, 모습, 사진, 출처, 유튜브, 캡처, 드라마..."
3409,삼성전자,5930,매일경제,2021091217,[부음] 최영자씨 별세 외,http://news.mk.co.kr/newsRead.php?no=879709&ye...,최영자씨별세윤종덕삼성전자커뮤니케이션팀전무윤종수LG생활건강HDBSCM팀장윤은주씨모친상...,2021-09-13,17,1,"[최영자, 윤종덕, 성전, 커뮤니케이션, 전무, 윤종수, 생활, 건강, 윤은주, 모..."


In [76]:
test = giro_test(test)

okt 실행 완료
태그 삭제 완료
품사태깅 완료
명사, 형용사 추출 완료
불용어 사전 로딩 완료
불용어 처리 완료
단어 길이 1이하 단어 제거
test 데이터 전처리 완료


In [77]:
test

,st_n,st_cd,news,date,title,url,text,date_d,date_h,ud,Contents_input
0,삼성전자,5930,매일경제,2018010106,미디어 세상을 접수한 유튜브를 파헤치다,http://premium.mk.co.kr/view.php?no=21031,경제경영BookCafe블루오션시프트김위찬르네마보안비즈니스북스블루오션전략이후12년4차...,2018-01-02,06,1,"[경제, 경영, 블루, 오션, 시프트, 김위찬, 네마, 보안, 비즈니스, 북스, 블..."
1,삼성전자,5930,매일경제,2018010117,기반산업 따라 희비 갈린 부동산,http://news.mk.co.kr/newsRead.php?year=2018&no...,한때미분양의늪으로통하던경기도파주시부동산시장이달라지고있다지난해6월분양한파주문산역동문굿...,2018-01-02,17,1,"[미분, 통, 경기도, 파주시, 부동산, 시장, 지난해, 여명, 수도권, 광역급행,..."
2,삼성전자,5930,매일경제,2018010100,"동원산업·휠라코리아 ""생큐 트럼프""",http://news.mk.co.kr/newsRead.php?year=2018&no=30,2018법인세인하수혜올해1월부터법인세인하를골자로한미국세제개혁법안이현실화하면서국내증시...,2018-01-02,00,1,"[법인세, 인하, 수혜, 올해, 법인, 세율, 대폭, 핵심, 국외, 자금, 자국, ..."
3,삼성전자,5930,매일경제,2018010104,"[2018 국내 업종별 전망] 반도체, D램 독주 계속…`반도체 정점론` 부담",http://news.mk.co.kr/newsRead.php?year=2018&no...,인공지능AI사물인터넷IoT등4차산업혁명수요폭발에따른슈퍼사이클로한국경제의효자역할을톡톡...,2018-01-02,04,1,"[인공, 지능, 사물인터넷, 산업혁명, 수요, 폭발, 슈퍼, 사이클, 국경, 제의,..."
4,삼성전자,5930,매일경제,2018010104,[2018 경제기상도] `1월효과` 기대 커진 증시…반도체·바이오株 계속 달릴듯,http://news.mk.co.kr/newsRead.php?year=2018&no...,지난해연말조정장세는올해초코스피와코스닥시장1월효과의기폭제가될전망이다상장사들의탄탄한실적...,2018-01-02,04,1,"[지난해, 연말, 정장, 해초, 코스피, 코스닥시장, 돌파, 코스닥, 하이닉스, 정..."
...,...,...,...,...,...,...,...,...,...,...,...
3406,삼성전자,5930,매일경제,2021091216,세계 최대 삼성전자 프리미엄 가전 매장 통했다…10년만에 신규 백화점 갤러리아 광교...,http://news.mk.co.kr/newsRead.php?no=879535&ye...,갤러리아광교에자리잡은삼성프리미엄스토어내부모습사진제공갤러리아지난해3월갤러리아가10년만...,2021-09-13,16,1,"[갤러리아, 광교, 자리, 삼성, 프리미엄, 스토어, 내부, 모습, 사진, 제공, ..."
3407,삼성전자,5930,매일경제,2021091214,"""10만원에 페이스북 아마존 애플 넷플릭스 구글 다 살 수 있다?""",http://news.mk.co.kr/newsRead.php?no=878988&ye...,고가주식쪼개서매매아마존알파벳LG생활건강등주당100만원이훌쩍넘는고가우량주를커피한잔값에...,2021-09-13,14,1,"[가주, 매매, 아마존, 알파벳, 생활, 건강, 등주, 대의, 주식시장, 참여, 대..."
3408,삼성전자,5930,매일경제,2021091206,"펜트하우스 천서진 물로 씻은 그 폰, 갤Z플립3 수심 3m 수영장에 빠트렸더니…",http://news.mk.co.kr/newsRead.php?no=878229&ye...,천서진이피묻은갤럭시Z플립3를물로씻고작동하는모습사진출처유튜브캡처드라마펜트하우스3주인공...,2021-09-13,06,1,"[서진, 갤럭시, 플립, 를물, 작동, 모습, 사진, 출처, 유튜브, 캡처, 드라마..."
3409,삼성전자,5930,매일경제,2021091217,[부음] 최영자씨 별세 외,http://news.mk.co.kr/newsRead.php?no=879709&ye...,최영자씨별세윤종덕삼성전자커뮤니케이션팀전무윤종수LG생활건강HDBSCM팀장윤은주씨모친상...,2021-09-13,17,1,"[최영자, 윤종덕, 성전, 커뮤니케이션, 전무, 윤종수, 생활, 건강, 윤은주, 모..."


In [78]:
test['giro'] = 0
for index, wordList in enumerate(test['Contents_input']):
    for word in wordList:
        if word in pos_gi:
            test['giro'][index] += 1
        if word in neg_gi:
            test['giro'][index] -= 1

In [79]:
test

,st_n,st_cd,news,date,title,url,text,date_d,date_h,ud,Contents_input,giro
0,삼성전자,5930,매일경제,2018010106,미디어 세상을 접수한 유튜브를 파헤치다,http://premium.mk.co.kr/view.php?no=21031,경제경영BookCafe블루오션시프트김위찬르네마보안비즈니스북스블루오션전략이후12년4차...,2018-01-02,06,1,"[경제, 경영, 블루, 오션, 시프트, 김위찬, 네마, 보안, 비즈니스, 북스, 블...",-21
1,삼성전자,5930,매일경제,2018010117,기반산업 따라 희비 갈린 부동산,http://news.mk.co.kr/newsRead.php?year=2018&no...,한때미분양의늪으로통하던경기도파주시부동산시장이달라지고있다지난해6월분양한파주문산역동문굿...,2018-01-02,17,1,"[미분, 통, 경기도, 파주시, 부동산, 시장, 지난해, 여명, 수도권, 광역급행,...",3
2,삼성전자,5930,매일경제,2018010100,"동원산업·휠라코리아 ""생큐 트럼프""",http://news.mk.co.kr/newsRead.php?year=2018&no=30,2018법인세인하수혜올해1월부터법인세인하를골자로한미국세제개혁법안이현실화하면서국내증시...,2018-01-02,00,1,"[법인세, 인하, 수혜, 올해, 법인, 세율, 대폭, 핵심, 국외, 자금, 자국, ...",-2
3,삼성전자,5930,매일경제,2018010104,"[2018 국내 업종별 전망] 반도체, D램 독주 계속…`반도체 정점론` 부담",http://news.mk.co.kr/newsRead.php?year=2018&no...,인공지능AI사물인터넷IoT등4차산업혁명수요폭발에따른슈퍼사이클로한국경제의효자역할을톡톡...,2018-01-02,04,1,"[인공, 지능, 사물인터넷, 산업혁명, 수요, 폭발, 슈퍼, 사이클, 국경, 제의,...",5
4,삼성전자,5930,매일경제,2018010104,[2018 경제기상도] `1월효과` 기대 커진 증시…반도체·바이오株 계속 달릴듯,http://news.mk.co.kr/newsRead.php?year=2018&no...,지난해연말조정장세는올해초코스피와코스닥시장1월효과의기폭제가될전망이다상장사들의탄탄한실적...,2018-01-02,04,1,"[지난해, 연말, 정장, 해초, 코스피, 코스닥시장, 돌파, 코스닥, 하이닉스, 정...",-29
...,...,...,...,...,...,...,...,...,...,...,...,...
3406,삼성전자,5930,매일경제,2021091216,세계 최대 삼성전자 프리미엄 가전 매장 통했다…10년만에 신규 백화점 갤러리아 광교...,http://news.mk.co.kr/newsRead.php?no=879535&ye...,갤러리아광교에자리잡은삼성프리미엄스토어내부모습사진제공갤러리아지난해3월갤러리아가10년만...,2021-09-13,16,1,"[갤러리아, 광교, 자리, 삼성, 프리미엄, 스토어, 내부, 모습, 사진, 제공, ...",12
3407,삼성전자,5930,매일경제,2021091214,"""10만원에 페이스북 아마존 애플 넷플릭스 구글 다 살 수 있다?""",http://news.mk.co.kr/newsRead.php?no=878988&ye...,고가주식쪼개서매매아마존알파벳LG생활건강등주당100만원이훌쩍넘는고가우량주를커피한잔값에...,2021-09-13,14,1,"[가주, 매매, 아마존, 알파벳, 생활, 건강, 등주, 대의, 주식시장, 참여, 대...",-13
3408,삼성전자,5930,매일경제,2021091206,"펜트하우스 천서진 물로 씻은 그 폰, 갤Z플립3 수심 3m 수영장에 빠트렸더니…",http://news.mk.co.kr/newsRead.php?no=878229&ye...,천서진이피묻은갤럭시Z플립3를물로씻고작동하는모습사진출처유튜브캡처드라마펜트하우스3주인공...,2021-09-13,06,1,"[서진, 갤럭시, 플립, 를물, 작동, 모습, 사진, 출처, 유튜브, 캡처, 드라마...",-5
3409,삼성전자,5930,매일경제,2021091217,[부음] 최영자씨 별세 외,http://news.mk.co.kr/newsRead.php?no=879709&ye...,최영자씨별세윤종덕삼성전자커뮤니케이션팀전무윤종수LG생활건강HDBSCM팀장윤은주씨모친상...,2021-09-13,17,1,"[최영자, 윤종덕, 성전, 커뮤니케이션, 전무, 윤종수, 생활, 건강, 윤은주, 모...",1


In [80]:
len(test[(test.giro > 0) & (test.ud > 0)])/len(test[test.ud > 0])

0.41811175337186895

In [81]:
len(test[(test.giro < 0) & (test.ud < 0)])/len(test[test.ud < 0])

0.4886731391585761

In [82]:
hap = len(test[(test.giro < 0) & (test.ud < 0)])+len(test[(test.giro > 0) & (test.ud > 0)])

In [83]:
hap/len(test)

0.45646437994722955

In [84]:
len(test[test.giro==0])

303

In [85]:
len(test[test.ud ==0])

0

In [86]:
test.to_csv('./result/test_300.csv', index=False)

## ---------------- db----------------

In [ ]:
import pymysql
import pandas

db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='3.35.70.166', 
    db='proj', 
    charset = 'utf8'
)

cursor = db.cursor(pymysql.cursors.DictCursor)

sql = "select * from news_craw_005930 where length(date) = 10"  # date 널값 제외
cursor.execute(sql)

result = cursor.fetchall()

# 데이터 프레임으로 변경
df = pandas.DataFrame(result)

### ------------- ver2 실습 -----------------

## --------------------실습 ----------------------

## 함수 실습 _ ver1

In [ ]:
df_a, pos_cnt, nega_cnt, positive_r, negative_r =  giro_dic(df, 5930)

In [ ]:
df_a

## 실습 1

In [ ]:
df.head(1)

In [ ]:
df['text'].str.replace('[\n|\t]','')

In [ ]:
# pip install xlrd
# !pip install openpyxl

In [ ]:
df_hundai = pd.read_excel('./news_craw_hyundai.xlsx', engine = 'openpyxl')

In [ ]:
df_hundai = df_hundai.dropna()

In [ ]:
df_hundai['date'] = df_hundai['date'].astype(int)

In [ ]:
df_hundai.head()

In [ ]:
df_hundai.drop('Unnamed: 0',1, inplace= True)

In [ ]:
df_hundai.columns = ['st_n', 'st_cd', 'news', 'n_date', 'title', 'url', 'text']

In [ ]:
df_hundai.head(1)

In [ ]:
df_a.head()

In [ ]:
pos_cnt

In [ ]:
nega_cnt

In [ ]:
positive_r

In [ ]:
negative_r